##### NDES Seminar: Elements of Statistical Learning

## Chapter 4: Linear Methods for Classification

#### Summer semester 2019, April 2nd.

Collection of notebooks from Chapter 4: https://github.com/dgkim5360/the-elements-of-statistical-learning-notebooks/

Notebooks from https://github.com/dgkim5360/the-elements-of-statistical-learning-notebooks/tree/master/chapter04-linear-methods-for-classification
- - - -



**Outline:**

- **4.1** Introduction
- **4.2** Linear Regression of an Indicator Matrix
- **4.3**   Linear Discriminant Analysis
- **4.3.1**    Regularized Discriminant Analysis
- **4.3.2**    Computations for LDA
- **4.3.3**    Reduced-Rank Linear Discriminant Analysis
- **4.4**   Logistic Regression
- **4.4.1**    Fitting Logistic Regression Models
- **4.4.2**    Example: South African Heart Disease
- **4.4.3**    Quadratic Approximations and Inference
- **4.4.4**  L1Regularized Logistic Regression
- **4.4.5**    Logistic Regression or LDA?
- **4.5**   Separating Hyperplanes
- **4.5.1**    Rosenblatt’s Perceptron Learning Algorithm
- **4.5.2**    Optimal Separating Hyperplanes

### Some notebook magic

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import display, HTML, Markdown
display(HTML("<style>.container { width:98% !important; }</style>"))

### Imports

In [ ]:
import scipy
import scipy.linalg
import pandas as pd
import matplotlib.pyplot as plt

# $\S$ 4.1. Introduction

Since our predictor $G(x)$ takes values in a discrete set $\mathcal{G}$, we can always divide the input space into a collection of regions labeled according to the classification. We saw in Chapter 2 that the boundaries of these regions can be rough or smooth, depending on the prediction function. For an important class of procedures, these *decision boundaries* are linear; this is what we will mean by linear methodds for classification.

### Linear regression

In Chapter 2 we fit linear regression models to the class indicator variable, and classify to the largest fit. Suppose there are $K$ classes labeled $1,\cdots,K$, and the fitted linear model for the $k$th indicator response variable is

\begin{equation}
\hat{f}_k(x) = \hat\beta_{k0} + \hat\beta_k^Tx.
\end{equation}

The decision boundary between class $k$ and $l$ is that set of points

\begin{equation}
\left\lbrace x: \hat{f}_k(x) = \hat{f}_l(x) \right\rbrace = \left\lbrace x: (\hat\beta_{k0}-\hat\beta_{l0}) + (\hat\beta_k-\hat\beta_l)^Tx = 0 \right\rbrace,
\end{equation}

which is an affine set or hyperplane. Since the same is true for any pair of classes, the input space is divided into regions of constant classification, with piecewise hyperplanar decision boundaries.

### Discriminant function

The regression approach is a member of a class of methods that model *discriminant functions* $\delta_k(x)$ for each class, and then classify $x$ to the class with the largest value for its discriminant function. Methods that model the posterior probabilities $\text{Pr}(G=k|X=x)$ are also in this class. Clearly, if either the $\delta_k(x)$ or $\text{Pr}(G=k|X=x)$ are linear in $x$, then the decision boundaries will be linear.

### Logit transformation

Actually, all we require is that some monotone transformation of $\delta_k$  or $\text{Pr}(G=k|X=x)$ be linear for the decision boundaries to be linear. For example, if there are two classes, a popular model for the posterior probabilities is

\begin{align}
\text{Pr}(G=1|X=x) &= \frac{\exp(\beta_0+\beta^Tx)}{1+\exp(\beta_0+\beta^Tx)},\\
\text{Pr}(G=2|X=x) &= \frac{1}{1+\exp(\beta_0+\beta^Tx)},\\
\end{align}

where the monotone transformation is the *logit* transformation

\begin{equation}
\log\frac{p}{1-p},
\end{equation}

and in fact we see that

\begin{equation}
\log\frac{\text{Pr}(G=1|X=x)}{\text{Pr}(G=2|X=x)} = \beta_0 + \beta^Tx.
\end{equation}

The decision boundary is the set of points for which the *log-odds* are zero, and this is a hyperplane defined by

\begin{equation}
\left\lbrace x: \beta_0+\beta^Tx = 0 \right\rbrace.
\end{equation}

We discuss two very popular but different methods that result in linear log-odds or logits: Linear discriminant analysis and linear logistic regression. Although they differ in their derivation, the essential difference between them is in the way the lineaer function is fir to the training data.

### Separating hyperplanes

A more direct approach is to explicitly model the boundaries between the classes as linear. For a two-class problem, this amounts to modeling the decision boundary as a hyperplane; a normal vector and a cut-point.

We will look at two methods that explicitly look for "separating hyperplanes".
1. The well-known *perceptron* model of Rosenblatt (1958), with an algorithm that finds a separating hyperplane in the training data, if one exists.
2. Vapnik (1996) finds an *optimally separating hyperplane* if one exists, else finds a hyperplane that minimizes some measure of overlap in the training data.

We treat separable cases here, and defer the nonseparable case to Chapter 12.

### Scope for generalization

We can expand the input by including their squares $X_1^2,X_2^2,\cdots$, and cross-products $X_1X_2,\cdots$, thereby adding $p(p+1)/2$ additional variables. Linear functions in the augmented space map down to quadratic decision boundaires. FIGURE 4.1 illustrates the idea.

This approach can be used with any basis transformation $h(X): \mathbb{R}^p\mapsto\mathbb{R}^q$ with $q > p$, and will be explored in later chapters.

# $\S$ 4.2. Linear Regression of an Indicator Matrix

Here each of the response categories are coded via an indicator variable. Thus if $\mathcal{G}$ has $K$ classes, there will be $K$ such indicators $Y_k$, $k=1,\cdots,K$, with

\begin{equation}
Y_k = 1 \text{ if } G = k \text{ else } 0.
\end{equation}

These are collected together in a vector $Y=(Y_1,\cdots,Y_k)$, and the $N$ training instances of these form an $N\times K$ *indicator response matrix* $\mathbf{Y}$, which is a matrix of $0$'s and $1$'s, with each row having a single $1$.

We fit a linear regression model to each of the columns of $\mathbf{Y}$ simultaneously, and the fit is given by

\begin{equation}
\hat{\mathbf{Y}} = \mathbf{X}\left(\mathbf{X}^T\mathbf{X}\right)^{-1}\mathbf{X}^T\mathbf{Y} = \mathbf{X}\hat{\mathbf{B}}.
\end{equation}

Note that we have a coefficient vector for each response columns $\mathbf{y}_k$, and hence a $(p+1)\times K$ coefficient matrix $\hat{\mathbf{B}} = \left(\mathbf{X}^T\mathbf{X}\right)^{-1}\mathbf{X}^T\mathbf{Y}$. Here $\mathbf{X}$ is the model matrix with $p+1$ columns with a leading columns of $1$'s for the intercept.

A new observation with input $x$ is classified as follows:
* Compute the fitted output $\hat{f}(x)^T = (1, x^T)^T\hat{\mathbf{B}}$, a $K$ vector.
* Identify the largest component and classify accordingly:  

\begin{equation}
\hat{G}(x) = \arg\max_{k\in\mathcal{G}} \hat{f}_k(x).
\end{equation}

### Rationale

One rather formal justification is to view the regression as an estimate of conditional expectation. For the random variable $Y_k$, 

\begin{equation}
\text{E}(Y_k|X=x) = \text{Pr}(G=k|X=x),
\end{equation}

so conditional expectation of each $Y_k$ seems a sensible goal.

The real issue is: How good an approximation to conditional expectation is the rather rigid linear regression model? Alternatively, are the $\hat{f}_k(x)$ reasonable estimates of the posterior probabilities $\text{Pr}(G=k|X=x)$, and more importantly, does this matter?

It is quite straightforward to verify that, as long as the model has an intercept,

\begin{equation}
\sum_{k\in\mathcal{G}}\hat{f}_k(x) = 1.
\end{equation}

However it is possible that $\hat{f}_k(x) < 0$ or $\hat{f}_k(x) > 1$, and typically some are. This is a consequence of the rigid nature of linear regression, especially if we make predictions outside the hull of the training data. These violations in themselves do not guarantee that this approach will not work, and in fact on many problems it gives similar results to more standard linear methods for classification.

If we allow linear regression onto basis expansions $h(X)$ of the inputs, this approach can lead to consistent estimates of the probabilities. As the size of the training set $N$ grows bigger, we adaptively include more basis elements so that linear regression onto these basis functions approaches conditional expectation. We discuss such approaches in Chapter 5.

### A more simplistic viewpoint

Denote $t_k$ as the $k$th column of $\mathbf{I}_K$, the $K\times K$ identity matrix, then a more simplistic viewpoint is to construct *targets* $t_k$ for each class. The response vector ($i$th row of $\mathbf{Y}$)

\begin{equation}
y_i = t_k \text{ if } g_i = k.
\end{equation}

We might then fit the linear model by least squares: The criterion is a sum-of-squared Euclidean distances of the fitted vectors from their targets.

\begin{equation}
\min_{\mathbf{B}} \sum_{i=1}^N \left\| y_i - \left[ (1,x_i^T)\mathbf{B} \right]^T \right\|^2.
\end{equation}

Then a new observation is classified by computing its fitted vector $\hat{f}(x)$ and classifying to the closest target:

\begin{equation}
\hat{G}(x) = \arg\max_k \left\| \hat{f}(x)-t_k \right\|^2.
\end{equation}

This is exactly the same as the previous linear regression approach. Below are the reasons:

1. The sum-of-squared-norm criterion is exactly the same with multiple response linear regression, just viewed slightly differently. The component decouple and can be rearranged as a separate linear model for each element because there is nothing in the model that binds the diferent response together.

2. The closest target classification rule is exactly the same as the maximum fitted component criterion.

### Masked class with the regression approach

There is a serious problem with the regression approach when the number of class $K\ge 3$, especially prevalent when $K$ is large. Because of the rigid nature of the regression model, classes can be *masked* by others. FIGURE 4.2 illustrates an extreme situation when $K=3$. The three classes are perfectly separated by linear decision boundaries, yet linear regression misses the middle class completely.

In [ ]:
"""FIGURE 4.2. (Left) The data come from three classes in R^2 and are easily
separated by linear decision boundaries. This plot shows the boundaires
found by linear regression of the indicator response variables. The middle
class is completely masked (never dominates).

Instead of drawing the decision boundary, showing the classified data is
enough to illustrate masking phenomenon."""

# Make the simulation data
size_cluster = 300
mat_cov = scipy.eye(2)
cluster1 = scipy.random.multivariate_normal([-4, -4], mat_cov, size_cluster)
cluster2 = scipy.random.multivariate_normal([0, 0], mat_cov, size_cluster)
cluster3 = scipy.random.multivariate_normal([4, 4], mat_cov, size_cluster)
target1, target2, target3 = scipy.eye(3)
# print(target1, target2, target3, type(target1))
mat_x0 = scipy.concatenate((cluster1, cluster2, cluster3))
mat_x = scipy.hstack((scipy.ones((size_cluster*3, 1)), mat_x0))
mat_y = scipy.vstack((scipy.tile(target1, (size_cluster, 1)),
                      scipy.tile(target2, (size_cluster, 1)),
                      scipy.tile(target3, (size_cluster, 1))))
# print(mat_x, mat_x.shape)
# print(mat_y, mat_y.shape)

# Multiple linear regression
mat_beta = scipy.linalg.solve(mat_x.T @ mat_x, mat_x.T @ mat_y)
mat_y_hat = mat_x @ mat_beta
assert scipy.allclose(mat_y_hat.sum(axis=1), 1)
# print(mat_y_hat)
idx_classified_y = mat_y_hat.argmax(axis=1)
# print(idx_classified_y, idx_classified_y.size)

classified_cluster1 = mat_x0[idx_classified_y == 0]
classified_cluster2 = mat_x0[idx_classified_y == 1]
classified_cluster3 = mat_x0[idx_classified_y == 2]

In [ ]:
fig42 = plt.figure(0, figsize=(10, 5))
ax1 = fig42.add_subplot(1, 2, 1)
ax1.plot(cluster1[:,0], cluster1[:,1], 'o', color='C0', markersize=2)
ax1.plot(cluster2[:,0], cluster2[:,1], 'o', color='C1', markersize=2)
ax1.plot(cluster3[:,0], cluster3[:,1], 'o', color='C2', markersize=2)
ax1.set_xlabel('X_1')
ax1.set_ylabel('X_2')
ax1.set_title('Real Data')

ax2 = fig42.add_subplot(1, 2, 2)
ax2.plot(classified_cluster1[:,0], classified_cluster1[:,1], 'o', color='C0', markersize=2)
ax2.plot(classified_cluster2[:,0], classified_cluster2[:,1], 'o', color='C1', markersize=2)
ax2.plot(classified_cluster3[:,0], classified_cluster3[:,1], 'o', color='C2', markersize=2)
ax2.set_xlabel('X_1')
ax2.set_ylabel('X_2')
ax2.set_title('Classification using Linear Regression')
plt.show()

In [ ]:
"""FIGURE 4.3. The effects of masking on linear regression in R for a three
class problem.

The rug plot at the base indicates the positions and class membership of
each observation. The three curves in each panel are the fitted regression
to the three-class indicator variables.

We see on the left that the middle class line is horizontal and its fitted
values are never dominant! Thus, observations from class 2 are classified
either as class 1 or 3."""
# Make the simulation data
size_cluster = 300
cluster1 = scipy.random.normal(-4, size=(size_cluster,1))
cluster2 = scipy.random.normal(size=(size_cluster,1))
cluster3 = scipy.random.normal(4, size=(size_cluster,1))
target1, target2, target3 = scipy.eye(3)
# print(target1, target2, target3, type(target1))
mat_x0 = scipy.concatenate((cluster1, cluster2, cluster3))
mat_x = scipy.hstack((scipy.ones((size_cluster*3, 1)), mat_x0))
mat_y = scipy.vstack((scipy.tile(target1, (size_cluster, 1)),
                      scipy.tile(target2, (size_cluster, 1)),
                      scipy.tile(target3, (size_cluster, 1))))

# Multiple linear regression with degree 1
mat_beta = scipy.linalg.solve(mat_x.T @ mat_x, mat_x.T @ mat_y)
mat_y_hat = mat_x @ mat_beta
idx_classified_y = mat_y_hat.argmax(axis=1)

In [ ]:
fig43 = plt.figure(1, figsize=(10, 5))
ax1 = fig43.add_subplot(1, 2, 1)
ax1.plot(mat_x0, mat_y_hat[:, 0], 'o', color='C0', markersize=2)
ax1.plot(mat_x0, mat_y_hat[:, 1], 'o', color='C1', markersize=2)
ax1.plot(mat_x0, mat_y_hat[:, 2], 'o', color='C2', markersize=2)
y_floor, _ = ax1.get_ylim()
ax1.plot(cluster1, [y_floor]*size_cluster, 'o', color='C0', markersize=2)
ax1.plot(cluster2, [y_floor]*size_cluster, 'o', color='C1', markersize=2)
ax1.plot(cluster3, [y_floor]*size_cluster, 'o', color='C2', markersize=2)
ax1.set_title('Degree = 1')

# Multiple linear regression with degree 2
mat_x2 = scipy.hstack((mat_x, mat_x0*mat_x0))
mat_beta2 = scipy.linalg.solve(mat_x2.T @ mat_x2, mat_x2.T @ mat_y)
mat_y2_hat = mat_x2 @ mat_beta2

ax2 = fig43.add_subplot(1, 2, 2)
ax2.plot(mat_x0, mat_y2_hat[:, 0], 'o', color='C0', markersize=2)
ax2.plot(mat_x0, mat_y2_hat[:, 1], 'o', color='C1', markersize=2)
ax2.plot(mat_x0, mat_y2_hat[:, 2], 'o', color='C2', markersize=2)
y_floor, _ = ax2.get_ylim()
ax2.plot(cluster1, [y_floor]*size_cluster, 'o', color='C0', markersize=2)
ax2.plot(cluster2, [y_floor]*size_cluster, 'o', color='C1', markersize=2)
ax2.plot(cluster3, [y_floor]*size_cluster, 'o', color='C2', markersize=2)
ax2.set_title('Degree = 2')
plt.show()

For this simple example a quadratic rather than linear fit would solve the problem. However, if there were 4 classes, a quadratic would not come down fast enough, and a cubic would be needed as well. A loose but general rule is that if $K\ge 3$ classes are lined up, polynomial terms up to degree $K-1$ might be needed to resolve them.

Note also that these are polynomials along the derived direction passing through the centroids, which can have orbitrary orientation. So in $p$-dimensional input space, one would need general polynomial terms and cross-products of total degree $K-1$, $O(p^{K-1})$ terms in all, to resolve such worst-case scenarios.

The example is extreme, but for large $K$ and small $p$ such maskings natrually occur. As a more realistic illustration, FIGURE 4.4 is a projection of the training data for a vowel recognition problem onto an informative two-dimensional subspace.

In [ ]:
"""FIGURE 4.4. A two-dimensional plot of the vowel training data.

There are K=11 classes in p=10 dimensions, and this is the best view in
terms of a LDA model (Section 4.3.3). The heavy circles are the projected
mean vectors for each class. The class overlap is considerable.

This is a difficult classficiation problem, and the best methods achieve
around 40% errors on the test data. The main point here is summarized in
Table 4.1; masking has hurt in this case.

Even though the textbook may used the first 2 principal components, here
simply the first 2 coordinates x.1 and x.2 are used."""
vowel_df = pd.read_csv('../data/vowel/vowel.train', index_col=0)
df_y = vowel_df['y']
df_x2d = vowel_df[['x.1', 'x.2']]
df_x2d.describe()
vowel_df.head()

In [ ]:
grouped = df_x2d.groupby(df_y)

fig44 = plt.figure(2, figsize=(10, 10))
ax = fig44.add_subplot(1, 1, 1)

for y, x in grouped:
    x_mean = x.mean()
    color = next(ax._get_lines.prop_cycler)['color']
    ax.plot(x['x.1'], x['x.2'], 'o', color=color)
    ax.plot(x_mean[0], x_mean[1], 'o', color=color, markersize=10,
            markeredgecolor='black', markeredgewidth=3)
ax.set_xlabel('Coordinate 1 for Training Data')
ax.set_ylabel('Coordinate 2 for Training Data')
ax.set_title('Linear Discriminant Analysis')
plt.show()

# $\S$ 4.3. Linear Discriminant Analysis

[$\S$ 2.4. Decision theory for classification](/notebooks/chapter2-overview-of-supervised-learning/section4-statistical-decision-theory.ipynb) tells us that we need to know the class posteriors $\text{Pr}(G|X)$ for optimal classification. Suppose
* $f_k(x)$ is the class-conditional density of $X$ in class $G=k$,
* $\pi_k$ is the prior probability of class $k$, with $\sum\pi_k=1$.

A simple application of Bayes theorem gives us

\begin{equation}
\text{Pr}(G=k|X=x) = \frac{f_k(x)\pi_k}{\sum_{l=1}^K f_l(x)\pi_l}.
\end{equation}

We see that in terms of ability to classify, it is enough to have the $f_k(x)$.

Many techniques are based on models for the class densities:
* linear and quadratic discriminant analysis use Gaussian densities;
* more flexible mixtures of Gaussian allow for nonlinear decision boundaires ($\S$ 6.8);
* general nonparametric density estimates for each class density allow the most flexibility ($\S$ 6.6.2);
* *Naive Bayes* models are a variant of the previous case, and assume that the inputs are conditionally independent in each class; i.e., each of the class densities are products of marginal densities ($\S$ 6.6.3).

### LDA from multivariate Gaussian

Suppose that we model each class density as multivariate Gaussian

\begin{equation}
f_k(x) = \frac{1}{(2\pi)^{p/2}|\Sigma_k|^{1/2}}\exp\left\lbrace -\frac{1}{2}(x-\mu_k)^T\Sigma_k^{-1}(x-\mu_k) \right\rbrace
\end{equation}

Linear discriminant analysis (LDA) arises in the special case when we assume that the classes have a common covariance matrix $\Sigma_k=\Sigma,\forall k$.

In comparing two classes $k$ and $l$, it is sufficient to look at the log-ratio, and we see that as an equation linear in $x$,

\begin{align}
\log\frac{\text{Pr}(G=k|X=x)}{\text{Pr}(G=l|X=x)} &= \log\frac{f_k(x)}{f_l(x)} + \log\frac{\pi_k}{\pi_l} \\
&= \log\frac{\pi_k}{\pi_l} - \frac{1}{2}\mu_k^T\Sigma^{-1}\mu_k + \frac{1}{2}\mu_l^T\Sigma^{-1}\mu_l + x^T\Sigma^{-1}(\mu_k-\mu_l) \\
&= \delta_k(x) - \delta_l(x),
\end{align}

where $\delta_k$ is the *linear discriminant function*

\begin{equation}
\delta_k(x) = x^T\Sigma^{-1}\mu_k - \frac{1}{2}\mu_k^T\Sigma^{-1}\mu_k + \log\pi_k.
\end{equation}

This linear log-odds function implies that the decision boundary between classes $k$ and $l$

\begin{equation}
\left\lbrace x: \delta_k(x) - \delta_l(x) = 0 \right\rbrace
\end{equation}

is linear in $x$; in $p$ dimensions a hyperplane. Also the linear discriminant functions are equivalent description of the decision rule, with

\begin{equation}
G(x) = \arg\max_k \delta_k(x).
\end{equation}

In [ ]:
"""FIGURE 4.5. An idealized example with K=3, p=2, and a common covariance

Here the right panel shows the LDA classification results instead of the
decision boundaries."""

In [ ]:
size_cluster = 30
# mat_rand = scipy.rand(2, 2)
# cov = mat_rand.T @ mat_rand / 10
cov = scipy.array([[1.0876306, 0.2065698],
                   [0.2065698, 0.1157603]])/2

cluster1 = scipy.random.multivariate_normal([-.5, 0], cov, size_cluster)
cluster2 = scipy.random.multivariate_normal([.0, .0], cov, size_cluster)
cluster3 = scipy.random.multivariate_normal([0, .5], cov, size_cluster)

# Estimating parameters
vec_mean1 = cluster1.mean(axis=0)
vec_mean2 = cluster2.mean(axis=0)
vec_mean3 = cluster3.mean(axis=0)

cluster_centered1 = cluster1 - vec_mean1
cluster_centered2 = cluster2 - vec_mean2
cluster_centered3 = cluster3 - vec_mean3
mat_cov = (cluster_centered1.T @ cluster_centered1 +
           cluster_centered2.T @ cluster_centered2 +
           cluster_centered3.T @ cluster_centered3)/(3*size_cluster-3)

In [ ]:
# Calculate linear discriminant scores
sigma_inv_mu123 = scipy.linalg.solve(
    mat_cov,
    scipy.vstack((vec_mean1, vec_mean2, vec_mean3)).T,
)
sigma_inv_mu1, sigma_inv_mu2, sigma_inv_mu3 = sigma_inv_mu123.T

mat_x = scipy.vstack((cluster1, cluster2, cluster3))
mat_delta = (mat_x @ sigma_inv_mu123 -
             scipy.array((vec_mean1 @ sigma_inv_mu1,
                          vec_mean2 @ sigma_inv_mu2,
                          vec_mean3 @ sigma_inv_mu3))/2)

cluster_classified1 = mat_x[mat_delta.argmax(axis=1) == 0]
cluster_classified2 = mat_x[mat_delta.argmax(axis=1) == 1]
cluster_classified3 = mat_x[mat_delta.argmax(axis=1) == 2]

In [ ]:
fig45 = plt.figure(0, figsize=(10, 5))
ax1 = fig45.add_subplot(1, 2, 1)
ax1.plot(cluster1[:, 0], cluster1[:, 1], 'o', color='C0')
ax1.plot(cluster2[:, 0], cluster2[:, 1], 'o', color='C1')
ax1.plot(cluster3[:, 0], cluster3[:, 1], 'o', color='C2')
ax1.plot(-.5, 0, 'o', color='C0', markersize=10, markeredgecolor='black',
         markeredgewidth=3)
ax1.plot(.5, 0, 'o', color='C1', markersize=10, markeredgecolor='black',
         markeredgewidth=3)
ax1.plot(0, .5, 'o', color='C2', markersize=10, markeredgecolor='black',
         markeredgewidth=3)
ax1.set_title('Simulation Data')

ax2 = fig45.add_subplot(1, 2, 2)
ax2.plot(cluster_classified1[:, 0], cluster_classified1[:, 1], 'o', color='C0')
ax2.plot(cluster_classified2[:, 0], cluster_classified2[:, 1], 'o', color='C1')
ax2.plot(cluster_classified3[:, 0], cluster_classified3[:, 1], 'o', color='C2')
ax2.plot(vec_mean1[0], vec_mean1[1], 'o', color='C0', markersize=10,
         markeredgecolor='black', markeredgewidth=3)
ax2.plot(vec_mean2[0], vec_mean2[1], 'o', color='C1', markersize=10,
         markeredgecolor='black', markeredgewidth=3)
ax2.plot(vec_mean3[0], vec_mean3[1], 'o', color='C2', markersize=10,
         markeredgecolor='black', markeredgewidth=3)
ax2.set_title('LDA Results')
plt.show()

In [ ]:
print(
    'Proportion of observations the model classified correctly: '
    + str(
        sum(mat_delta.argmax(axis=1) == scipy.hstack(([0] * len(cluster1), [1] * len(cluster2), [2] * len(cluster3)))) / len(mat_delta)
    )
)

### Estimating parameters

In practice we do not know the parameters of the Gaussian distributions, and will need to estimate them using our training data:
* $\hat\pi_k = N_k/N$,
* $\hat\mu_k = \sum_{g_i = k} x_i/N_k$;
* $\hat\Sigma = \sum_{k=1}^K \sum_{g_i=k}(x_i-\hat\mu_k)(x_i-\hat\mu_k)^T/(N-K)$.

### Simple correspondence between LDA and linear regression with two classes

The LDA rule classifies to class 2 if

\begin{equation}
x^T\hat\Sigma^{-1}(\hat\mu_2-\hat\mu_1) > \frac{1}{2}(\hat\mu_2+\hat\mu_1)^T\hat\Sigma^{-1}(\hat\mu_2+\hat\mu_1) - \log\frac{N_2}{N_1},
\end{equation}

and class 1 otherwise. If we code the targets in the two classees as $+1$ and $-1$ respectively, then the coefficient vector from least squares is proportional to the LDA direction shown above (Exercise 4.2). However unless $N_1=N_2$ the intercepts are different and hence the resulting decision rules are different.

If $K>2$, LDA is not the same as linear regression of the class indicator matrix, and it avoids the masking problems (Hastie et al., 1994). A correspondence can be established through the notion of *optimal scoring*, discussed in $\S$ 12.5.

### Practice beyond the Gaussian assumption

Since the derivation of the LDA direction via least squares does not use a Gaussian assumption for the features, its applicability extends beyond the realm of Gaussian data. However the derivation of the particular intercept or cut-point given in the above LDA rule *does* require Gaussian data. Thus it makes sense to instead choose the cut-point that empirically minimizes training error for a given dataset. This something we have found to work well in practive, but have not seen it mentioned in the literature.

### Quadratic Discriminant Analysis

If the $\Sigma_k$ are not assumed to be equal, then the convenient cancellations do not occur. We then get *quadratic discriminant functions* (QDA),

\begin{equation}
\delta_k(x) = -\frac{1}{2}\log|\Sigma_k| -\frac{1}{2}(x-\mu_k)^T\Sigma_k^{-1}(x-\mu_k) + \log\pi_k
\end{equation}

The decision boundary between each pair of classes $k$ and $l$ is described by a quadratic equation $\left\lbrace x: \delta_k(x) = \delta_l(x) \right\rbrace$.

In [ ]:
"""FIGURE 4.6. QDA and augmented LDA

Two methods for fitting quadratic boundaries. The differences are generally
small; QDA is the preferred approach, with the LDA method a convenient
substitute."""
print('Under construction ...')

This estimates for QDA are similar to those for LDA, except that separate covariance matrices must be estimated for each class. When $p$ is large this can mean a dramatic increase in parameters.

> Since the decision boundaries are functions of the parameters of the densities, counting the number of parameters must be done with care.

For LDA, it seems there are $(K-1)\times(p+1)$ paramters, since we only need the differences $\delta_k(x)-\delta_K(x)$ between the discriminant functions where $K$ is some pre-chosen class (here the last), and each difference requires $p+1$ parameters. Likewise for QDA there will be $(K-1)\times\lbrace p(p+3)/2+1 \rbrace$ parameters.

Both LDA and QDA perform well on an amazingly large and diverse set of classification tasks. See the textbook (the page 111) for the evidence (Michie et al., 1994).

### Why LDA and QDA have such a good track record?

The data are approximately Gaussian, or for LDA the covariances are approximately equal? Maybe not.

More likely a reason is that the data can only support simple decision boundaries such as linear or quadratic, and the estimates provided via the Guassian models are stable.

This is a bias-variance tradeoff -- we can put up with the bias of a linear decision boundary because it can be estimated with much lower variance than more exotic alternatives. This argument is less believable for QDA, since it can have many parameters itself, although perhaps fewer than the non-parametric alternatives.

## $\S$ 4.3.1. Regularized Discriminant Analysis

### $\Sigma_k \leftrightarrow \Sigma$
These methods are very similar in flavor to ridge regression. Friedman (1989) proposed a compromise between LDA and QDA, which allows one to shrink the separate covariances of QDA toward a common covariance $\hat\Sigma$ as in LDA. The regularized covariance matrices have the form

\begin{equation}
\hat\Sigma_k(\alpha) = \alpha\hat\Sigma_k + (1-\alpha)\hat\Sigma,
\end{equation}

where $\alpha\in[0,1]$ allows a continuum of models between LDA and QDA, and needs to be specified. In practice $\alpha$ can be chosen based on the performance of the model on validation data, or by cross-validation.

### $\Sigma \leftrightarrow \sigma$

Similar modifications allow $\hat\Sigma$ itelf to be shrunk toward the scalar covariance,

\begin{equation}
\hat\Sigma(\gamma) = \gamma\hat\Sigma + (1-\gamma)\hat\sigma^2\mathbf{I},
\end{equation}

for $\gamma\in[0,1]$.

Combining two regularization leads to a more general family of covariances $\hat\Sigma(\alpha,\gamma)$.

### To be continued

In Chapter 12, we discuss other regularized versions of LDA, which are more suitable when the data arise from digitized analog signals and images. In these situations the features are high-dimensional and correlated, and the LDA coefficients can be regularized to be smooth or sparse in original domain of the signal.

In Chapter 18, we also deal with very high-dimensional problems, where for example, the features are gene-expression measurements in microarray studies.

## $\S$ 4.3.2. Computations for LDA

Computations for LDA and QDA are simplified by diagonalizing $\hat\Sigma$ or $\hat\Sigma_k$. For the latter, suppose we compute the eigen-decomposition, for each $k$,

\begin{equation}
\hat\Sigma_k = \mathbf{U}_k\mathbf{D}_k\mathbf{U}_K^T,
\end{equation}

where $\mathbf{U}_k$ is $p\times p$ orthogonal, and $\mathbf{D}_k$ a diagonal matrix of positive eigenvalues $d_{kl}$.

Then the ingredients for $\delta_k(x)$ are
* $(x-\hat\mu_k)^T\hat\Sigma_k^{-1}(x-\hat\mu_k) = \left[\mathbf{U}_k(x-\hat\mu_k)\right]^T\mathbf{D}_k^{-1}\left[\mathbf{U}_k(x-\hat\mu_k)\right]$
* $\log|\hat\Sigma_k| = \sum_l \log d_{kl}$

Note that the inversion of diagonal matrices only requires elementwise reprocials.

The LDA classifier can be implemented by the following pair of steps:
* *Sphere* the data w.r.t. the common covariance estimate $\hat\Sigma = \mathbf{U}\mathbf{D}\mathbf{U}^T$:  
\begin{equation}
X* \rightarrow \mathbf{D}^{-\frac{1}{2}}\mathbf{U}^TX,
\end{equation}  
The common covariance estimate of $X*$ will now be the identity.
* Classify to the closest class centroid in the transformed space, modulo the effect of the class prior probabilities $\pi_k$.

## $\S$ 4.3.3. Reduced-Rank Linear Discriminant Analysis

The $K$ centroids in $p$-dimensional input space lie in an affine subspace of dimension $\le K-1$, and if $p \gg K$, then there will possibly be a considerable drop in dimension. Part of the popularity of LDA is due to such an additional restriction that allows us to view informative low-dimensional projections of the data.

Moreover, in locating the closest centroid, we can ignore distances orthogonal to this subspace, since they will contribute equally to each class. Thus we might just as well project the $X^*$ onto this centroid-spanning subspace $H_{K-1}$, and make distance comparisons there.

Therefore there is a fundamental dimension reduction in LDA, namely, that we need only consider the data in a subspace of dimension at most $K-1$. If  $K=3$, e.g., this could allow us to view the data in $\mathbb{R}^2$, color-coding the classes. In doing so we would not have relinquished any of the information needed for LDA classification.

### What if $K>3$? Principal components subspace

We might then ask for a $L<K-1$ dimensional subspace $H_L \subseteq H_{K-1}$ optimal for LDA in some sense. Fisher defined optimal to mean that the projected centroids were spread out as much as possible in terms of variance. This amounts to finding principal component subspaces of the centroids themselves ($\S$ 3.5.1, $\S$ 14.5.1).

In FIGURE 4.4 with the vowel data, there are eleven classes, each a different vowel sound, in a 10D input space. The centroids require the full space in this case, since $K-1=p$, but we have shown an optimal 2D subspace.

The dimensions are ordered, so we can compute additional dimensions in sequence. FIGURE 4.8 shows four additional pairs of coordinates, a.k.a. *canonical* or *discriminant* variables.

In summary then, finding the sequences of optimal subspaces for LDA involves the following steps:
* Compute the $K\times p$ matrix of class centroids $\mathbf{M}$  
the common covariance matrix $\mathbf{W}$ (for *within*-class covariance).
* Compute $\mathbf{M}^* = \mathbf{MW}^{-\frac{1}{2}}$ using the eigen-decomposition of $\mathbf{W}$.
* Compute $\mathbf{B}^*$, the covariance matrix of $\mathbf{M}^*$ ($\mathbf{B}$ for *between*-class covariance),  
and its eigen-decomposition $\mathbf{B}^* = \mathbf{V}^*\mathbf{D}_B\mathbf{V}^{*T}$.  
The columns $v_l^*$ of $\mathbf{V}^*$ in sequence from first to last define the coordinates of the optimal subspaces.
* Then the $l$th *discriminant variable* is given by  
\begin{equation}
Z_l = v_l^TX \text{ with } v_l = \mathbf{W}^{-\frac{1}{2}}v_l^*.
\end{equation}

In [ ]:
"""FIGURE 4.8. Four projections onto pairs of canonical variates.

"""
df_vowel = pd.read_csv('../data/vowel/vowel.train', index_col=0)
print('A pandas DataFrame of size {} x {} '
      'has been loaded.'.format(*df_vowel.shape))
df_vowel.head()

In [ ]:
df_y = df_vowel.pop('y')
mat_x = df_vowel.as_matrix()

In [ ]:
df_x_grouped = df_vowel.groupby(df_y)
size_class = len(df_x_grouped)
df_mean = df_x_grouped.mean()

In [ ]:
def within_cov(df_grouped: pd.DataFrame,
               df_mean: pd.DataFrame)->scipy.ndarray:
    """Compute the within-class covariance matrix"""
    size_class = len(df_grouped)
    dim = df_mean.columns.size
    mat_cov = scipy.zeros((dim, dim))
    n = 0
    
    for (c, df), (_, mean) in zip(df_grouped, df_mean.iterrows()):
        n += df.shape[0]
        mat_centered = (df - mean).as_matrix()
        mat_cov += mat_centered.T @ mat_centered
    return mat_cov/(n-size_class)

In [ ]:
mat_M = df_mean.as_matrix()
mat_W = within_cov(df_x_grouped, df_mean)
vec_D, mat_U = scipy.linalg.eigh(mat_W)
# print(scipy.allclose(mat_U @ scipy.diag(vec_D) @ mat_U.T, mat_W))

mat_W_inv_sqrt = (mat_U @ scipy.diag(scipy.sqrt(scipy.reciprocal(vec_D))) @
                  mat_U.T)
mat_Mstar = mat_M @ mat_W_inv_sqrt    
vec_Mstar_mean = mat_Mstar.mean(axis=0)

mat_Mstar_centered = mat_Mstar - vec_Mstar_mean
mat_Bstar = mat_Mstar_centered.T @ mat_Mstar_centered/(mat_Mstar.shape[0]-1)

vec_DBstar, mat_Vstar = scipy.linalg.eigh(mat_Bstar)

mat_V = mat_W_inv_sqrt @ mat_Vstar
mat_x_canonical = mat_x @ mat_V

In [ ]:
fig48 = plt.figure(0, figsize=(10, 10))

ax11 = fig48.add_subplot(2, 2, 1)
ax12 = fig48.add_subplot(2, 2, 2)
ax21 = fig48.add_subplot(2, 2, 3)
ax22 = fig48.add_subplot(2, 2, 4)

for y in range(1, size_class+1):
    mat_x_grouped = mat_x_canonical[df_y == y]
    c = next(ax11._get_lines.prop_cycler)['color']
    ax11.plot(mat_x_grouped[:, -1], mat_x_grouped[:, -3], 'o', color=c)
    ax12.plot(mat_x_grouped[:, -2], mat_x_grouped[:, -3], 'o', color=c)
    ax21.plot(mat_x_grouped[:, -1], mat_x_grouped[:, -7], 'o', color=c)
    ax22.plot(mat_x_grouped[:, -9], mat_x_grouped[:, -10], 'o', color=c)
    
    vec_centroid = mat_x_grouped.mean(axis=0)
    ax11.plot(vec_centroid[-1], vec_centroid[-3], 'o', color=c, markersize=10, markeredgecolor='black', markeredgewidth=3)
    ax12.plot(vec_centroid[-2], vec_centroid[-3], 'o', color=c, markersize=10, markeredgecolor='black', markeredgewidth=3)
    ax21.plot(vec_centroid[-1], vec_centroid[-7], 'o', color=c, markersize=10, markeredgecolor='black', markeredgewidth=3)
    ax22.plot(vec_centroid[-9], vec_centroid[-10], 'o', color=c, markersize=10, markeredgecolor='black', markeredgewidth=3)
    
ax11.set_xlabel('Coordinate 1')
ax11.set_ylabel('Coordinate 3')
ax12.set_xlabel('Coordinate 2')
ax12.set_ylabel('Coordinate 3')
ax21.set_xlabel('Coordinate 1')
ax21.set_ylabel('Coordinate 7')
ax22.set_xlabel('Coordinate 9')
ax22.set_ylabel('Coordinate 10')
fig48.suptitle('Linear Discriminant Analysis')
plt.show()

### Maximize between-class variance relative to within-class

Fisher arrived at this decomposition via a different route, without referencing to Gaussian distributions at all. He posed the problem:

> Find the linear combination $Z=a^TX$ such that the between-class variance is maximized relative to the within-class variance.

FIGURE 4.9 shows why this criterion makes sense. Although the direction joining the centroids separates the means as much as possible (i.e., maximizes the between-class variance), there is considerable overlap between the projected classes due to the nature of the covariances. By taking the covariance into account as well, a direction with minimum overlap can be found.

The between-class variance of Z is $a^T\mathbf{B}a$ and the within-class variance $a^T\mathbf{W}a$ and the *total* covariance $\mathbf{T} = \mathbf{B} + \mathbf{W}$, ignoring class information. Then Fisher's problem  amounts to maximizing the *Rayleigh quotient*,

\begin{equation}
\max_a \frac{a^T\mathbf{B}a}{a^T\mathbf{W}a},
\end{equation}

or equivalently

\begin{equation}
\max_a a^T\mathbf{B}a \text{ subject to } a^T\mathbf{W}a = 1.
\end{equation}

This is a generalized eigenvalue problem, with $a$ given by the largest eigenvalue of $\mathbf{W}^{-1}\mathbf{B}$.

### Algorithm for the generalized eigenvalue problem

It is not hard to show (Exercise 4.1) the followings.
1. The optimal $a_1$ is identical to $v_1$ defined above.
2. Similarly one can find the next direction $a_2$, orthogonal in $\mathbf{W}$ to $a_1$, such that $a_2^T\mathbf{B}a_2/a_2^T\mathbf{W}a_2$ is maximized; the solution is $a_2 = v_2$, and so on.

The $a_l$ are referred to as *discriminant coordinates* or *canonical variates*, since an alternative derivation of these results is through a canonical correlation analysis of the indicator response matrix $\mathbf{Y}$ on the predictor matrix $\mathbf{X}$ ($\S$ 12.5).

### Summary

* Gaussian classification with common covariance leads to linear decision boundaries. Classficiation can be achieved by sphering the data w.r.t. $\mathbf{W}$, and classifying to the closest centroid (modulo $\log\pi_k$) in the sphered space.
* Since only the relative distances to the centroids count, one can confine the data to the subspace spanned by the centroids in the sphered space.
* This subspace can be further decomposed into successively optimal subspaces in terms of centroid separation. This decomposition is identical to the decomposition due to Fisher.

### Dimension reduction for classification

The reduced subspaces have been motivated as a data reduction (for viewing) tool. Can they also be used for classification, and what is the rationale?

Clearly they can, as in our original derivation; we simply limit the distance-to-centroid calculations to the chosen subspace. One can show that this is a Gaussian classfication rule with the additional restriction that the centroids of the Gaussian lie in a $L$-dimensional subspace of $\mathbb{R}^p$. Fitting such a model by maximum likelihood, and then constructing the posterior probabilities using Bayes' theorem amounts to the classification rule described above (Exercise 4.8).

### Impact of prior information $\pi_k$

Gaussian classification dictates the $\log\pi_k$ correction factor in the distance calculation. The reason for this correction can be seen in FIGURE 4.9. The misclassfication rate is based on the area of overlap between the two densities. If the $\pi_k$ are equal, then the optimal cut-point is midway between the projected means. If not equal, moving the cut-point toward the *smaller* class will improve the error rate. One can derive the linear rule using LDA (or any other method), and then choose the cut-point to minimize misclassification error over the training data.

In [ ]:
"""FIGURE 4.11. The decision boundaries for the classifier based on the 2D
LDA solution.

As an example of the benefit of the reduced-rank restriction, we return to
the vowel data with 11 classes and 10 variables, and hence 10 possible
dimensions for the classifier.
"""
fig411 = plt.figure(1, figsize=(10, 5))

ax1 = fig411.add_subplot(1, 2, 1)
ax2 = fig411.add_subplot(1, 2, 2)

mat_centroid2d = []
for y in range(1, size_class+1):
    mat_x2d_grouped = mat_x_canonical[df_y == y][:, -1:-3:-1]
    c = next(ax1._get_lines.prop_cycler)['color']
    ax1.plot(mat_x2d_grouped[:, 0], mat_x2d_grouped[:, 1], 'o', color=c)
    
    vec_centroid2d = mat_x2d_grouped.mean(axis=0)
    mat_centroid2d.append(vec_centroid2d)
    ax1.plot(vec_centroid2d[0], vec_centroid2d[1], 'o', color=c,
             markersize=10, markeredgecolor='black', markeredgewidth=3)

mat_centroid2d = scipy.array(mat_centroid2d)
vec_classified = scipy.array([
    ((mat_centroid2d - vec_x)**2).sum(axis=1).argmin()
    for vec_x in mat_x_canonical[:, -1:-3:-1]
])
for y, centroid in enumerate(mat_centroid2d):
    mat_x2d_classified = mat_x_canonical[vec_classified == y][:, -1:-3:-1]
    c = next(ax2._get_lines.prop_cycler)['color']
    ax2.plot(mat_x2d_classified[:, 0], mat_x2d_classified[:, 1], 'o',
             color=c)
    ax2.plot(centroid[0], centroid[1], 'o', color=c,
             markersize=10, markeredgecolor='black', markeredgewidth=3)

ax1.set_xlabel('Canonical Coordinate 1')
ax1.set_ylabel('Canonical Coordinate 2')
ax1.set_title('Projected Data')
ax2.set_xlabel('Canonical Coordinate 1')
ax2.set_ylabel('Canonical Coordinate 2')
ax2.set_title('Classification in Reduced Subspace')
plt.show()

### Connection between Fisher's reduced-rank discriminant analysis and regression of an indicator response matrix

It turns out that LDA amounts to the regression followed by an eigen-decomposition of $\hat{\mathbf{Y}}^T\mathbf{Y}$. In the case of two classes, there is a single discriminant variable that is identical up to a scalar multiplication to either of the columns of $\hat{\mathbf{Y}}$. A related fact is that if one transforms the original predictors $\mathbf{X}$ to $\hat{\mathbf{Y}}$, then LDA using $\hat{\mathbf{Y}}$ is identical to LDA in the original space (Exercise 4.3).

# $\S$ 4.4. Logistic Regression

The logistic regression model arises from the desire to model the posterior probabilities of the $K$ classes via linear functions in $x$, ensuring the natural properties of the probability: They sum to one and remain in $[0,1]$.

The model has the form

\begin{align}
\log\frac{\text{Pr}(G=1|X=x)}{\text{Pr}(G=K|X=x)} &= \beta_{10} + \beta_1^Tx \\
\log\frac{\text{Pr}(G=2|X=x)}{\text{Pr}(G=K|X=x)} &= \beta_{20} + \beta_2^Tx \\
&\vdots \\
\log\frac{\text{Pr}(G=K-1|X=x)}{\text{Pr}(G=K|X=x)} &= \beta_{(K-1)0} + \beta_{K-1}^Tx \\
\end{align}

The model is specified in terms of $K-1$ log-odds or logit transformations, reflecting the constraint that the probabilities sum to one. The choice of denominator ($K$ in this case) is arbitrary in that the estimates are equivalent under this choice.

### Sum to one

To emphasize the dependence on the entire parameter set $\theta = \left\lbrace \beta_{10}, \beta_1^T, \cdots, \beta_{(K-1)0}, \beta_{K-1}^T\right\rbrace$, we denote the probabilities

\begin{equation}
\text{Pr}(G=k|X=x) = p_k(x;\theta)
\end{equation}

A simple calculation shows that

\begin{align}
\text{Pr}(G=k|X=x) &= \frac{\exp(\beta_{k0}+\beta_k^Tx)}{1+\sum_{l=1}^{K-1}\exp(\beta_{l0}+\beta_l^Tx)}, \text{ for } k=1,\cdots,K-1, \\
\text{Pr}(G=K|X=x) &= \frac{1}{1+\sum_{l=1}^{K-1}\exp(\beta_{l0}+\beta_l^Tx)},
\end{align}

and they clearly sum to one.

When $K=2$, this model is especially simple, since there is only a single linear function.

## $\S$ 4.4.1. Fitting Logistic Regression Models

### Maximum likelihood

Logistic regression models are usually fit by maximum likelihood, using the conditional likelihood of $G$ given $X$. Since $\text{Pr}(G|X)$ completely specifies the conditional distribution, the *multinomial* distribution is appropriate.

The log-likelihood for $N$ observation is

\begin{equation}
l(\theta) = \sum_{i=1}^N \log p_{g_i}(x_i;\theta),
\end{equation}

where $p_k(x_i;\theta) = \text{Pr}(G=k|X=x_i;\theta)$

### Maximum likelihood for $K=2$ case

We discuss in detail the two-class case, sine the algorithms simplify considerably. It is convenient to code the two-class $g_i$ via a $0/1$ response $y_i$, where $y_i=1$ when $g_i=1$, and $0$ otherwise. Then we can let

\begin{align}
p_1(x;\theta) &= p(x;\theta), \\
p_2(x;\theta) &= 1- p(x;\theta). \\
\end{align}

The log-likelihood can be written

\begin{align}
l(\beta) &= \sum_{i=1}^N \left\lbrace y_i\log p(x_i;\beta) + (1-y_i)\log(1-p(x_i;\beta)) \right\rbrace \\
&= \sum_{i=1}^N \left\lbrace y_i\beta^Tx_i - \log(1+\exp(\beta^Tx)) \right\rbrace,
\end{align}

where $\beta^T = \lbrace \beta_{10}, \beta_1^T \rbrace$, and we assume that the vector of inputs $x_i$ includes the constant term 1 to acommodate the intercept.

To maximize the log-likelihood, we set its derivatives to zero. These *score* equations are

\begin{equation}
\frac{\partial l(\beta)}{\partial\beta} = \sum_{i=1}^N x_i(y_i-p(x_i;\beta)) = 0,
\end{equation}

which are $p+1$ equations *nonlinear* in $\beta$. Notice that since $x_{i1} =1$, the first score equation specifies that

\begin{equation}
\sum_{i=1}^N y_i = \sum_{i=1}^N p(x_i;\beta),
\end{equation}

implying that the *expected* number of class ones matches the observed number (and hence also class twos).

### Newton-Raphson algorithm

To solve the score equation, we use the Newton-Raphson algorithm, which requires the second-derivative or Hessian matrix

\begin{equation}
\frac{\partial^2 l(\beta)}{\partial\beta\partial\beta^T} = -\sum_{i=1}^N x_ix_i^T p(x_i;\beta)(1-p(x_i;\beta)).
\end{equation}

Starting with $\beta^{\text{old}}$, a single Newton update is

\begin{equation}
\beta^{\text{new}} = \beta^{\text{old}} - \left( \frac{\partial^2 l(\beta)}{\partial\beta\partial\beta^T} \right)^{-1} \frac{\partial l(\beta)}{\partial\beta},
\end{equation}

where the derivatives are evaluated at $\beta^{\text{old}}$.

### The same thing in matrix notation

Let
* $\mathbf{y}$ be the vector of $y_i$ values,
* $\mathbf{X}$ the $N\times (p+1)$ matrix of $x_i$ values,
* $\mathbf{p}$ the vector of fitted probabilities with $i$th element $p(x_i;\beta^{\text{old}})$, and
* $\mathbf{W}$ $N\times N$ diagonal matrix of weights with $i$th diagonal elements $p(x_i;\beta^{\text{old}})(1-p(x_i;\beta^{\text{old}}))$.

Then we have

\begin{align}
\frac{\partial l(\beta)}{\partial\beta} &= \mathbf{X}^T(\mathbf{y}-\mathbf{p}) \\
\frac{\partial^2l(\beta)}{\partial\beta\partial\beta^T} &= -\mathbf{X}^T\mathbf{WX},
\end{align}

and thus the Newton step is

\begin{align}
\beta^{\text{new}} &= \beta^{\text{old}} + (\mathbf{X}^T\mathbf{WX})^{-1}\mathbf{X}^T(\mathbf{y}-\mathbf{p}) \\
&= (\mathbf{X}^T\mathbf{WX})^{-1} \mathbf{X}^T\mathbf{W}\left( \mathbf{X}\beta^{\text{old}} + \mathbf{W}^{-1}(\mathbf{y}-\mathbf{p}) \right) \\
&= (\mathbf{X}^T\mathbf{WX})^{-1}\mathbf{X}^T\mathbf{W}\mathbf{z},
\end{align}

where we have re-expressed the Newton step as weighted least squares step, with the response

\begin{equation}
\mathbf{z} = \mathbf{X}\beta^{\text{old}} + \mathbf{W}^{-1}(\mathbf{y}-\mathbf{p}),
\end{equation}

sometimes known as the *adjusted response*.

### Iteratively reweighted least squares

These equations for the Newton step get solved repeatedly, since at each iteration $p$ changes, and hence so does $\mathbf{W}$ and $\mathbf{z}$. This algorithm is referred to as *iteratively reweighted least squares* or IRLS, since each iteration solves the weighted least squares problem:

\begin{equation}
\beta^{\text{new}} \leftarrow \arg\min_\beta (\mathbf{z}-\mathbf{X}\beta)^T\mathbf{W}(\mathbf{z}-\mathbf{X}\beta)
\end{equation}

It seems that $\beta=0$ is a good starting value, although convergence is never guaranteed. Typically the algorithm does converge, since the log-likelihood is concave, but overshooting can occur. In the rare cases that the log-likelihood decreases, step size halving will guarantee convergence.

### Multiclass case with $K\ge 3$

The Newton step can also be expressed as an IRLS, but with a *vector* of $K-1$ responses and a nondiagonal weight matrix per observation. (Exercise 4.4)

Alternatively coordinate-descent methods ($\S$ 3.8.6) can be used to maximize the log-likelihood efficiently.

The $\textsf{R}$ package $\textsf{glmnet}$ (Friedman et al., 2010) can fit very large logistic regression problems efficiently, both in $N$ and $p$.

### Goal of logistic regression

Logistic regression models are used mostly as a data analysis and inference tool, where the goal is to understand the role of the input variables in *explaning* the outcome. Typically many models are fit in a search for a parsimonious model involving a subset of the variables, possibly with some interactions terms. The following example illustrates some of the issues involved.

## $\S$ 4.4.2. Example: South African Heart Disease

The data in FIGURE 4.12 are a subset of the Coronary Risk-Factor Study (CORIS) baseline survey, carried out in three rural areas of the Western Cape, South Africa (Rousseauw et al., 1983). These data are described in more detail in Hastie and Tibshirani (1987).

In [ ]:
"""FIGURE 4.12. A scatterplot matrix of the South African heart desease
data.

"""

In [ ]:
df_saheart = pd.read_csv('../data/heart/SAheart.data', index_col=0)
print('A pandas DataFrame of size {} x {} '
      'has been loaded.'.format(*df_saheart.shape))
df_saheart.head()

In [ ]:
df_saheart.pop('adiposity')
df_saheart.pop('typea')  # The textbook doesn't use them
df_y = df_saheart.pop('chd')

In [ ]:
df_saheart['famhist'] = df_saheart['famhist'].map({'Present': 1,
                                                   'Absent': 0})
df_saheart.describe()

In [ ]:
colors = df_y.apply(lambda y: 'C0' if y else 'C1')
pd.plotting.scatter_matrix(df_saheart, c=colors, figsize=(10, 10))
plt.show()

We fit a logistic-regression model by maximum likelihood, giving the results shown in TABLE 4.2.

In [ ]:
"""TABLE 4.2. Results from a logistic regression fit to the South African
heart disease data
"""
mat_X = df_saheart.as_matrix()
size_training, size_predictor = mat_X.shape
size_beta = size_predictor + 1

vec_y = df_y.as_matrix()
mat_1X = scipy.hstack((scipy.ones((size_training, 1)), mat_X))


def fvec_p(mat_x:scipy.ndarray, vec_beta:scipy.ndarray)->scipy.ndarray:
    num = scipy.exp(mat_x@vec_beta)
    return num/(num+1)


def fdiag_W(mat_x:scipy.ndarray, vec_beta:scipy.ndarray)->scipy.ndarray:
    vec_p = fvec_p(mat_x, vec_beta)
    return vec_p*(1-vec_p)

In [ ]:
vec_beta_old = scipy.zeros(size_beta)
vec_increment = scipy.ones(size_beta)
while (vec_increment**2).sum() > 1e-8:
    vec_p = fvec_p(mat_1X, vec_beta_old)
    gradient = mat_1X.T @ (vec_y-vec_p)
    hessian = mat_1X.T @ scipy.diag(fdiag_W(mat_1X, vec_beta_old)) @ mat_1X
    vec_increment = scipy.linalg.solve(hessian, gradient)
    vec_beta_new = vec_beta_old + vec_increment
    vec_beta_old = vec_beta_new.copy()

In [ ]:
"""I have no idea where the Std. Error comes from..."""
print('{0:>15} {1:>15} {2:>15} {3:>15}'.format('Term', 'Coefficient',
                                               'Std. Error', 'Z Score'))
print('-'*64)
table_term = ['intercept'] + list(df_saheart.columns)
for term, coeff in zip(table_term, vec_beta_new):
    print('{0:>15} {1:>15f}'.format(term, coeff))

This summary includes $Z$ scores ($\frac{\text{coefficients}}{\text{stderr}}$); a nonsignificant $Z$ score suggests a coefficient can be dropped from the model. Each of these correspond formally to a test of the null hypothesis that the coefficient in question is zero, while all the others are not (a.k.a. the Wald test).

### Correlations between predictors

There are some surprises in this table, which must be interpreted with caution. Systolic blood pressure ($\textsf{sbp}$) is not significant! Nor is $\textsf{obesity}$, and its sign is negative.

This confusion is a result of the correlation between the set of predictors. On their own, both $\textsf{sbp}$ and $\textsf{obesity}$ are significant, However, in the presense of many other correlated variables, thery are no longer needed (and can even get a negative sign).

### Model selection

At this stage the analyst might do some model selection; find a subset of the variables that are sufficient for explaining their joint effect on the prevalence of the response ($\textsf{chd}$).

One way to proceed by is to drop the least significant coefficient, and refit the model. This is done repeatedly until no further terms can be dropped. This gave the model shown in TABLE 4.3.

A better but time-consuming strategy is to refit each of the models with one variable removed, and then perform an *analysis of deviance* to decide which variable to exclude.

The residual deviance of a fitted model is

\begin{equation}
\text{residual deviance}(\beta) = -2\text{ log-likelihood}(\beta),
\end{equation}

and the deviance between two models is the difference of their residual deviance, as

\begin{equation}
\text{deviance}(\beta^{(1)}, \beta^{(2)}) = \text{residual deviance}(\beta^{(1)}) - \text{residual deviance}(\beta^{(2)}).
\end{equation}

This strategy gave the same final model as TABLE 4.3.

### Interpretation of a coefficient

How does one interpret $\textsf{tobacco}$ coefficient of $0.081$ ($\text{Std. Error} = 0.026$), for example?

An increase of $1\text{kg}$ in lifetime tobacco usage accounts for an increase in the odds of coronary heart disease of $\exp(0.081)=1.084$ or $8.4\%$.

Incorporating the standard error we get an approximate $95\%$ confidence interval of

\begin{equation}
\exp(0.081 \pm 2\times 0.026) = (1.03, 1.14).
\end{equation}

We see that some of the variables have nonlinear effects, and when modeled appropriately, are not excluded from the model.

## $\S$ 4.4.3. Quadratic Approximations and Inference

Please check this section later...

## $\S$ 4.4.1. L1 Regularized Logistic Regression

Please check this section later...

## $\S$ 4.4.5. Logistic Regression or LDA?
### Common linearity

LDA has the log-posterior odds which are linear functions of x:

\begin{align}
\log\frac{\text{Pr}(G=k|X=x)}{\text{Pr}(G=K|X=x)} &= \log\frac{\pi_k}{\pi_K} - \frac{1}{2}(\mu_k-\mu_K)^T\Sigma^{-1}(\mu_k-\mu_K) + x^T\Sigma^{-1}(\mu_k-\mu_K) \\
&= \alpha_{k0} + \alpha_k^Tx,
\end{align}

and this linearity is a consequence of the Gaussian assumption for the class densities with a common covariance matrix.

The linear logistic model by construction has linear logits:

\begin{equation}
\log\frac{\text{Pr}(G=k|X=x)}{\text{Pr}(G=K|X=x)} = \beta_{k0} + \beta_k^Tx
\end{equation}

It seems that the models are the same, and they have the common logit-linear form for the posterior probabilities:

\begin{equation}
\text{Pr}(G=k|X=x) = \frac{\exp(\beta_{k0}+\beta_k^Tx)}{1+\sum_{l=1}^{K-1}\exp(\beta_{l0}+\beta_l^Tx)}
\end{equation}

### Different assumptions

Although they have exactly the same form, the difference lies in the way the linear coefficients are estimated: The logistic regression model is more general, in that it makes less assumptions.

Note the *joint density* of $X$ and $G$ as

\begin{equation}
\text{Pr}(X,G=k) = \text{Pr}(X)\text{Pr}(G=k|X),
\end{equation}

where $\text{Pr}(X)$ denotes the marginal density of the inputs $X$.

The logistic regression model leaves $\text{Pr}(X)$ as an arbitrary density function, and fits the parameters of $\text{Pr}(G|X)$ by maximizing the *conditional likelihood* -- the multinomial likelihood with probabilities the $\text{Pr}(G=k|X)$. Although $\text{Pr}(X)$ is totally ignored, we can think of this marginal density as being estimated in a fully nonparametric and unrestricted fashion, using empirical distribution function which places mass $1/N$ at each observation.

LDA fits the parameters by maximizing the full log-likelihood, based on the joint density

\begin{equation}
\text{Pr}(X,G=k) = \phi(X;\mu_k,\Sigma)\pi_k,
\end{equation}

where $\phi$ is the Gaussian density function. Standard normal theory leads easily to the estimates $\hat\mu_k$, $\hat\Sigma$, and $\hat\pi_k$ given in Section 4.3. Since the linear parameters of the logistic form

\begin{equation}
\log\frac{\text{Pr}(G=k|X=x)}{\text{Pr}(G=K|X=x)} = \log\frac{\pi_k}{\pi_K} - \frac{1}{2}(\mu_k-\mu_K)^T\Sigma^{-1}(\mu_k-\mu_K) + x^T\Sigma^{-1}(\mu_k-\mu_K)
\end{equation}

are functions of the Gaussian parameters, we get their maximum-likelihood estimates by plugging in the corresponding estimates.

### Role of the marginal density $\text{Pr}(X)$ in LDA

However, unlike in the conditional case, the marginal density $\text{Pr}(X)$ does play a role here. It is a mixture density

\begin{equation}
\text{Pr}(X) = \sum_{k=1}^K \pi_k\phi(X;\mu_k,\Sigma),
\end{equation}

which also involves the parameters. What role can this additional component or restriction play?

By relying on the additional model assumptions, we have more information about the parameters, and hence can estimate them more efficiently (lower variance). If in fact the true $f_k(x)$ are Gaussian, then in the worst case ignoring this marginal part of the likelihood constitutes a loss of efficiency of about $30\%$ asymptotically in the error rate (Efron, 1975). Paraphrasing: With $30\%$ more data, the conditional likelihood will do as well.

For example, observations far from the decision boundary (which are down-weighted by logistic regression) play a role in estimating the common covariance matrix. This is not a good news, because it also means that LDA is not robust to gross outliers.

### Marginal likelihood as a regularizer

The marginal likelihood can be thought of as a regularizer, requiring in some sense that class densities be *visible* from this marginal view.

For example, if the data in a two-class logistic regression model can be perfectly separated by a hyperplane, the maximum likelihood estimates of the parameters are undefined (i.e., infinite; see Exercise 4.5).

The LDA coefficients for the same data will be well defined, since the marginal likelihood will not permit these degeneracies.

### In real world

> It is generally felt that logistic regression is a safer and more robust bet than the LDA model, relying on fewer assumptions.

In practice these assumptions are never correct, and often some of the components of $X$ are qualitative variables. It is our experience that the models give very similar results, even when LDA is used inappropriately, such as with qualitative predictors.

# $\S$ 4.5. Separating Hyperplanes

We describe separating hyperplane classifiers, constructing linear decision boundaries that explicitly try to separate the data into different classes as well as possible. They provide the basis for support vector classifiers, discussed in Chapter 12.

FIGURE 4.14 shows 20 data points of two classes in $\mathbb{R}^2$, which can be separated by a linear boundary but there are infinitely many possible *separating hyperplanes*.

The orange line is the least squares solution to the problem, obtained by regressing the $-1/1$ response $Y$ on $X$ with intercept; the line is given by

\begin{equation}
\left\lbrace x: \hat\beta_0 + \hat\beta_1x_1 + \hat\beta_2x_2=0 \right\rbrace.
\end{equation}

This least squares solution does not do a perfect job in separating the points, and makes one error. This is the same boundary found by LDA, in light of its equivalence with linear regression in the two-class case ($\S$ 4.3 and Exercise 4.2).

### Perceptrons

Classifiers such as above, that compute a linear combination of the input features and return the sign, were called *perceptrons* in the engineering literatur in the late 1950s (Rosenblatt, 1958). Perceptrons set he foundations for the neural network models of the 1980s and 1990s.

### Review on vector algebra

FIGURE 4.15 depicts a hyperplane or *affine set* $L$ defined by the equation

\begin{equation}
f(x) = \beta_0 + \beta^T x = 0,
\end{equation}

since we are in $\mathbb{R}^2$ this is a line.

Here we list some properties:
1. For any two points $x_1$ and $x_2$ lying in $L$,  
\begin{equation}
\beta^T(x_1-x_2)=0,
\end{equation}
and hence the unit vector $\beta^* = \beta/\|\beta\|$ is normal to the surface of $L$.
2. For any point $x_0$ in $L$,  
\begin{equation}
\beta^Tx_0 = -\beta_0.
\end{equation}
3. The signed distance of any point $x$ to $L$ is given by  
\begin{align}
\beta^{*T}(x-x_0) &= \frac{1}{\|\beta\|}(\beta^Tx+\beta_0) \\
&= \frac{1}{\|f'(x)\|}f(x).
\end{align}
Hence $f(x)$ is proportional to the signed distance from $x$ to the hyperplane defined by $f(x)=0$.

## $\S$ 4.5.1. Rosenblatt's Perceptron Learning Algorithm

> The *perceptron learning algorithm* tries to find a separating hyperplane by minimizing the distance of misclassified points to the decision boundary.

If a response $y_i=1$ is misclassified, then $x_i^T\beta + \beta_0 \lt 0$, and the opposite for a misclassified response with $y_i=-1$. The goal is to minimize

\begin{equation}
D(\beta,\beta_0) = -\sum_{i\in\mathcal{M}} y_i(x_i^T\beta + \beta_0),
\end{equation}

where $\mathcal{M}$ indexes the set of misclassified points. The quantity is non-negative and proportional to the distance of the misclassified points to the decision boundary defined by $\beta^Tx+\beta_0=0$.

Assuming $\mathcal{M}$ is fixed, the gradient is given by

\begin{align}
\partial\frac{D(\beta,\beta_0)}{\partial\beta} &= -\sum_{i\in\mathcal{M}} y_ix_i, \\
\partial\frac{D(\beta,\beta_0)}{\partial\beta_0} &= -\sum_{i\in\mathcal{M}} y_i.
\end{align}

### Stochastic gradient descent

The algorithm in face uses *stochastic gradient descent* to minimize this piecewise linear criterion. This means that rather than computing the sum of the gradient contributions of each observation followed by a step in the negative gradient direction, a step in taken after each observation is visited.

Hence the misclassified observations are visited in some sequence, and the parameters $\beta$ are updated via

\begin{equation}
\begin{pmatrix}\beta \\ \beta_0\end{pmatrix}
\leftarrow
\begin{pmatrix}\beta \\ \beta_0\end{pmatrix}
+
\rho \begin{pmatrix}y_ix_i \\ y_i\end{pmatrix},
\end{equation}

where $\rho$ is the learning rate, which in this case can be taken to be $1$ WLOG.

If the classes are linearly separable, it can be shown that the algorithm converges to a separating hyperplane in a finite number of steps (Exercise 4.6). FIGURE 4.14 shows two solutions to a toy problem, each started at a different random guess.

In [ ]:
"""FIGURE 4.14. A toy example with two classes separable by a hyperplane.

The orange line is the least squares solution, which misclassifies one of
the training points. Also shown are two blue separating hyperplanes found
by the perceptron learning algorithm with different random starts.
"""
print('Under construction ...')

### Issues

There are a number of problems with this algorithm, summarized in Ripley (1996):
* When the data are separable, there are many solutions, and which one is found depends on the starting values.
* The "finite" number of steps can be very large. The smaller the gap, the longer the time to find it.
* When the data are not separable, the algorithm will not converge, and cycles develop. The cycles can be long and therefore hard to detect.

The second problem can often be eliminated by seeking a hyperplane not in the orignal space, but in a much enlarged space obtained by creating many basis-function transformations of the original variables. This is analogous to driving the residuals in a ploynomial regression problem down to zero by making the degree sufficiently large.

Perfect separation cannot always be achieved: For example, if observations from two different classes share the same input. It may not be desirable either, since the resulting model is likely to be overfit and will not generalizes well.

A rather elegant solution to the first problem is to add additional constraints to the separating hyperplane.

## $\S$ 4.5.2. Optimal Separating Hyperplanes

The *optimal separating hyperplanes* separates the two classes and maximizes the distance to the closest point from either class (Vapnik, 1996). Not only does this provide a unique solution to the separating hyperplane problem, but by maximizing the margin between two classes on the training data, this leads to better classification performance on test data.

### Formulation

We need to generalize the perceptron criterion

\begin{equation}
D(\beta,\beta_0) = -\sum_{i\in\mathcal{M}} y_i(x_i^T\beta + \beta_0).
\end{equation}

Consider the optimization problem

\begin{equation}
\max_{\beta,\beta_0,\|\beta\|=1} M \\
\text{subject to } y_i(x_i^T\beta + \beta_0) \ge M, \text{ for } i = 1,\cdots,N.
\end{equation}

The set of conditions ensure that all the points are at least a signed distance $M$ from the decision boundary defined by $\beta$ and $\beta_0$, and we seek the largest such $M$ and associated parameters.

We can get rid of the $\|\beta\| = 1$ constraint by replacing the conditions with

\begin{equation}
\frac{1}{\|\beta\|} y_i(x_i^T\beta + \beta_0) \ge M, \\
\text{or equivalently} \\
y_i(x_i^T\beta + \beta_0) \ge M\|\beta\|,
\end{equation}

which redefines $\beta_0$.

Since for any $\beta$ and $\beta_0$ satisfying these inequalities, any positively scaled multiple satisfies them too, we can arbitrarily set

\begin{equation}
\|\beta\| = \frac{1}{M},
\end{equation}

which leads to the equivalent formulation as

\begin{equation}
\min_{\beta,\beta_0} \frac{1}{2}\|\beta\|^2 \\
\text{subject to } y_i(x_i^T\beta + \beta_0) \ge 1, \text{ for }i=1,\cdots,N.
\end{equation}

In light of $(4.40)$, the constraints define an empty slab or margin around the linear decision boundary of thickness $1/\|\beta\|$. Hence we choose $\beta$ and $\beta_0$ to maximize its thickness.

### Convex optimization

This is a convex optimization problem (quadratic criterion with linear inequality constraints). The Lagrange (primal) function, to be minimized w.r.t. $\beta$ and $\beta_0$, is

\begin{equation}
L_P = \frac{1}{2}\|\beta\|^2 - \sum_{i=1}^N \alpha_i \left[ y_i(x_i^T\beta + \beta_0) -1 \right].
\end{equation}

Setting the derivatives to zero, we obtain:

\begin{align}
\beta &= \sum_{i=1}^N \alpha_i y_i x_i, \\
0 &= \sum_{i=1}^N \alpha_i y_i,
\end{align}

and substitutig these in $L_P$ we obtain the so-called Wolfe dual

\begin{equation}
L_D = \sum_{i=1}^N \alpha_i - \frac{1}{2} \sum_{i=1}^N \sum_{k=1}^N \alpha_i \alpha_k y_i y_k x_i^T x_k \\
\text{subject to } \alpha_i \ge 0 \text{ and } \sum_{i=1}^N \alpha_i y_i = 0.
\end{equation}

The solution is obtained by maximizing $L_D$ in the positive orthant, a simpler convex optimization problem, for which standard software can be used. In addition the solution must satisfy the Karush-Kuhn-Tucker (KKT) conditions, which includes the above three conditions and

\begin{equation}
\alpha_i \left[ y_i (x_i^T\beta + \beta_0) \right] = 0, \forall i.
\end{equation}

### Implications of the algorithm

From these we can see that
* if $\alpha_i \gt 0$, then $y_i(x_i^T\beta + \beta_0) = 1$, or in other words, $x_i$ is on the boundary of the slab;
* if $y_i(x_i^T\beta + \beta_0) \gt 1$, $x_i$ is not on the boundary of the slab, and $\alpha_i = 0$.

From the above condition of the primal Lagrange function

\begin{equation}
\beta = \sum_{i=1}^N \alpha_i y_i x_i,
\end{equation}

we see that the solution vector $\beta$ is defined in terms of a linear combination of the *support points* $x_i$ -- those points defined to be on the boundary of slab via $\alpha_i \gt 0$.

FIGURE 4.16 shows the optimal separating hyperplane for our toy example; these are three support points. Likewise, $\beta_0$ is obtained by solving the last KKT condition

\begin{equation}
\alpha_i \left[ y_i (x_i^T\beta + \beta_o) \right] = 0,
\end{equation}

for any of the support points.

In [ ]:
"""FIGURE 4.16. Optimal separating hyperplane

The same data aas in FIGURE 4.14. The shaded region delineates the maximum
margin separating the two classes. There are three support points
indicated, which lie on the boundary of the margin, and the optimal
separating hyperplane (blue line) bisects the slab. Included in the figure
is the boundary found using logistic regreesion (red line), which is very
close to the optimal separating hyperplane (see Section 12.3.3).

https://docs.scipy.org/doc/scipy/reference/optimize.html"""
print('Under construction (CVXOPT may be needed, priority low)...')

### Classification

The optimal separating hyperplane produces a function $\hat{f}(x) = x^T\hat\beta + \hat\beta_0$ for classifying new observations:

\begin{equation}
\hat{G}(x) = \text{sign } \hat{f}(x).
\end{equation}

Although none of the training observations fall in the margin (by construction), this will not necessarily be the case for test observations. The intuition is that a large margin on the training data will lead to good separation on the test data.

### Dependency on model assumption

The description of the solution in terms of support points seems to suggest that the optimal hyperplane focuses more on the points that count, and is more robust to model misspecification.

The LDA solution, on the other hand, depends on all of the data, even points far away from the decision boundary. Note, however, that the identification of these support points required the use of all the data. Of course, if the classes are really Gaussian, then LDA is optimal, and separating hyperplane will pay a price for focusing on the (noisier) data at the boundaries if the classes.

Included in FIGURE 4.16 is the logistic regression solution to this problem, fit by maximum likelihood. Both solutions are similar in this case. When a separating hyperplane exists, logistic regression will always find it, since the log-likelihood can be driven to $0$ in this case (Exercise 4.5).

*skipped*

### When the data are not separable

There will be no feasible solution to this problem, and an alternative formulation is needed.

Again one can enlarge the space using basis transformations, but this can lead to artificial separation through over-fitting. In Chapter 12 we discuss a more attractive alternative known as the *support vector machine*, which allows for overlap, but minimizes a measure of the extent of this overlap.